# Test Database API in Django

In [157]:
from movies.models import Star
from django.db.models import Avg, Count, Min, Max, Q, F, Value as V, Exists, OuterRef
from django.db.models.functions import Upper, Lower, Coalesce, Substr, NullIf, Extract

## Read entity : select *

In [116]:
star_set = Star.objects.all()
star_set

<QuerySet [<Star: Sylvester Stallone (1946-07-06 - 1)>, <Star: Steve McQueen (1960-10-09 - 2)>, <Star: Steve McQueen (1930-03-24 - 3)>, <Star: Jacqueline Bisset (1944-09-13 - 4)>, <Star: Alfred Hitchcock (1899-08-13 - 5)>]>

In [117]:
film_set = Film.objects.all()
film_set

<QuerySet [<Film: Joker (2019 - 1)>, <Film: Rambo: Last Blood (2019 - 2)>, <Film: Shame (2011 - 3)>, <Film: Bullitt (1968 - 4)>, <Film: Papillon (1973 - 5)>, <Film: L'Homme qui en savait trop (1956 - 6)>]>

## Projection : select annee

In [118]:
annees = Film.objects.values('annee')
annees

<QuerySet [{'annee': 2019}, {'annee': 2019}, {'annee': 2011}, {'annee': 1968}, {'annee': 1973}, {'annee': 1956}]>

In [119]:
annees_uniques = annees.distinct()
annees_uniques

<QuerySet [{'annee': 2019}, {'annee': 2011}, {'annee': 1968}, {'annee': 1973}, {'annee': 1956}]>

In [120]:
# annotate add the extra column which can be uses in a projection (or selection)
Film.objects.annotate(duree_0=Coalesce('duree',V(0))).values('duree_0')

<QuerySet [{'duree_0': 0}, {'duree_0': 89}, {'duree_0': 0}, {'duree_0': 114}, {'duree_0': 151}, {'duree_0': 120}]>

In [121]:
# with rename
Film.objects.values(title=F('titre'))

<QuerySet [{'title': 'Joker'}, {'title': 'Rambo: Last Blood'}, {'title': 'Shame'}, {'title': 'Bullitt'}, {'title': 'Papillon'}, {'title': "L'Homme qui en savait trop"}]>

## Selection : where annee = 2019

In [122]:
f = Film.objects.get(id=1)
f

<Film: Joker (2019 - 1)>

In [123]:
try:
    f = Film.objects.get(id=0)
except Film.DoesNotExist as e:
    print('No film with id 0 : ', e)

No film with id 0 :  Film matching query does not exist.


In [124]:
films_2019 = Film.objects.filter(annee=2019)
films_2019

<QuerySet [<Film: Joker (2019 - 1)>, <Film: Rambo: Last Blood (2019 - 2)>]>

In [125]:
stars_mcqueen = Star.objects.filter(nom='Steve McQueen')
stars_mcqueen

<QuerySet [<Star: Steve McQueen (1960-10-09 - 2)>, <Star: Steve McQueen (1930-03-24 - 3)>]>

In [126]:
# previous one is a shortcut for exact comparison
stars_mcqueen = Star.objects.filter(nom__exact='Steve McQueen')
stars_mcqueen

<QuerySet [<Star: Steve McQueen (1960-10-09 - 2)>, <Star: Steve McQueen (1930-03-24 - 3)>]>

In [127]:
# and filter + ignore case + attribute access
stars_mcqueen_1930 = Star.objects.filter(nom__iexact='steve mcqueen', date_naissance__year=1930)
stars_mcqueen_1930

<QuerySet [<Star: Steve McQueen (1930-03-24 - 3)>]>

In [128]:
# or filter + endswith comparison + Q function
stars_mcqueen_or_1944 = Star.objects.filter(Q(nom__endswith=' McQueen')|Q(date_naissance__year=1944))
stars_mcqueen_or_1944

<QuerySet [<Star: Steve McQueen (1960-10-09 - 2)>, <Star: Steve McQueen (1930-03-24 - 3)>, <Star: Jacqueline Bisset (1944-09-13 - 4)>]>

In [129]:
# startswith comparison (ignore case mode)
stars_steve = Star.objects.filter(nom__istartswith='steve ')
stars_steve

<QuerySet [<Star: Steve McQueen (1960-10-09 - 2)>, <Star: Steve McQueen (1930-03-24 - 3)>]>

In [130]:
# contains comparison (ignore case mode)
films_last = Film.objects.filter(titre__icontains='last')
films_last

<QuerySet [<Film: Rambo: Last Blood (2019 - 2)>]>

In [131]:
# regex matching (ignore case mode)
films_last = Film.objects.filter(titre__iregex=r'(^| )last($| )')
films_last

<QuerySet [<Film: Rambo: Last Blood (2019 - 2)>]>

In [132]:
# utiliser une fonction de calcul : Extract
# film dont le realisateur a 73 ans (dans l'année de ses 73 ans)
Film.objects.filter(annee=Extract('realisateur__date_naissance','year')+73)

<QuerySet [<Film: Rambo: Last Blood (2019 - 2)>]>

In [133]:
# add extra column and use it for filter
# NB : in this case, regex lookup is better
Film.objects.annotate(titre_3=Lower(Substr('titre',1,3))).filter(titre_3='bul')

<QuerySet [<Film: Bullitt (1968 - 4)>]>

## Aggregate functions

In [95]:
nb_films = Film.objects.count()
nb_films

5

In [96]:
nb_films_2019 = Film.objects.filter(annee=2019).count()
nb_films_2019

2

In [97]:
nb_films_by_annee = Film.objects.values('annee').annotate(nb_films=Count('id'))
nb_films_by_annee

<QuerySet [{'annee': 1968, 'nb_films': 1}, {'annee': 1973, 'nb_films': 1}, {'annee': 2011, 'nb_films': 1}, {'annee': 2019, 'nb_films': 2}]>

In [100]:
Film.objects.aggregate(Min('annee'), Max('annee'), duree_tot = Coalesce(Sum('duree'),V(0)))

{'duree_tot': 265, 'annee__min': 1968, 'annee__max': 2019}

In [102]:
Film.objects.exclude(realisateur=None).values('realisateur').annotate(Min('annee'), Max('annee'), duree_tot = Coalesce(Sum('duree'),V(0)))

<QuerySet [{'realisateur': 1, 'annee__min': 2019, 'annee__max': 2019, 'duree_tot': 0}, {'realisateur': 2, 'annee__min': 2011, 'annee__max': 2011, 'duree_tot': 0}]>

## Follow association

In [139]:
realisateur_film3 = Film.objects.get(id=3).realisateur
realisateur_film3

<Star: Steve McQueen (1960-10-09 - 2)>

In [135]:
acteurs_film4 = Film.objects.get(id=4).acteurs.all()
acteurs_film4

<QuerySet [<Star: Steve McQueen (1960-10-09 - 2)>, <Star: Jacqueline Bisset (1944-09-13 - 4)>]>

In [147]:
film_real_mcqueen = Film.objects.filter(realisateur__nom__iregex=r'(^| )mcqueen')
film_real_mcqueen

<QuerySet [<Film: Shame (2011 - 3)>]>

In [148]:
Film.objects.filter(acteurs__nom__iregex=r'(^| )mcqueen')

<QuerySet [<Film: Bullitt (1968 - 4)>, <Film: Papillon (1973 - 5)>]>

In [154]:
Film.objects.filter(acteurs__date_naissance__year__lt=1950)

<QuerySet [<Film: Rambo: Last Blood (2019 - 2)>, <Film: Bullitt (1968 - 4)>, <Film: Papillon (1973 - 5)>]>

In [165]:
Film.objects.filter(acteurs__date_naissance__year__lt=1940)

<QuerySet [<Film: Papillon (1973 - 5)>, <Film: Bullitt (1968 - 4)>]>

In [166]:
# same thing with exists
Film.objects.annotate(any_acteur_sub_1940=Exists(
    Star.objects.filter(date_naissance__year__lt=1940, id__in=OuterRef('acteurs')))) \
    .filter(any_acteur_sub_1940=True)

<QuerySet [<Film: Bullitt (1968 - 4)>, <Film: Papillon (1973 - 5)>]>